# Problema de Designação

O problema da designação é um tipo especial de problema de programação linear em que os **designados** estão sendo indicados para a realização de **tarefas**.

<img src="./img/designacao-grafo.png">

## Características do Problema
1. O número de designados e o número de tarefas é o mesmo. Esse número é representado por $n$
2. Deve-se atribuir a cada designado $i$ exatamente uma tarefa $j$.
3. Cada tarefa $j$ deve ser realizada exatamente por um designado $i$.
4. Há um custo $c_{ij}$ associado ao designado $i$ ($i = 1, 2, ... , n$) executando a tarefa $j$ ($j = 1,
2, ... , n$)
5. O objetivo é determinar como todas as $n$ designações devem ser feitas para minimizar o custo total.

## Modelo do Problema:

O modelo matemático para o problema da designação usa as seguintes variáveis de decisão:

\begin{equation}
  x_{ij} =
    \begin{cases}
      1 & \text{se o designado $i$ realiza a tarefa $j$}\\
      0 & \text{caso contrário}
    \end{cases}       
\end{equation}

para $i = 1, 2, ... , n$ e $j = 1, 2, ... , n$. Portanto, cada $x_{ij}$ é uma variável binária (ela possui valor O ou 1, sim ou não). Nesse caso, a decisão sim/não é: o designado $i$ deve executar a tarefa $j$?

Fazendo que $Z$ represente o custo total, o modelo para o problema da designação fica sendo:

Minimizar: $Z = \sum_{i=1}^n \sum_{j=1}^n c_{ij}x_{ij}$


Sujeito a: 

$\sum_{j=1}^n x_{ij} = 1$, para cada $j= 1,2,...,m$,

$\sum_{i=1}^m x_{ij} = 1$, para cada $i= 1,2,...,n$,

$x_{ij} \ge 0$, para todo $i$ e $j$.

$x_{ij}$ binário, para todo $i$ e $j$.

<img src="./img/designacao-tabela.png">

    O problema da designação é simplesmente um tipo especial de problema de transporte em que as origens agora são os designados e os destinos agora são as tarefas e nos quais

- Número de origens $m$ = número de destinos $n$ 
- Toda oferta $s_i = 1$,
- Toda demanda $d_j = 1$.

## Problema de exemplo

No exemplo, há cinco trabalhadores (numerados de 0 a 4) e quatro tarefas (numeradas de 0 a 3).

Os custos de atribuição de trabalhadores a tarefas são mostrados na tabela a seguir.

|   |0	|1	|2	|3  |
|---|---|---|---|---|
|0	|90	|80	|75	|70 |
|1	|35	|85	|55	|65 |
|2	|125|95	|90	|95 |
|3	|45	|110|95	|115|
|4	|50	|100|90	|100|

O problema é atribuir a cada trabalhador no máximo uma tarefa, sem que dois trabalhadores realizem a mesma tarefa, minimizando o custo total. Como há mais trabalhadores do que tarefas, um trabalhador não receberá tarefa.

In [1]:
#include <iostream>
#include <iomanip>
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

In [3]:
{
    // Data
    const std::vector<std::vector<double>> costs{
      {90, 80, 75, 70},   
      {35, 85, 55, 65},   
      {125, 95, 90, 95},
      {45, 110, 95, 115}, 
      {50, 100, 90, 100},
    };
    const int num_workers = costs.size();
    const int num_tasks = costs[0].size();

    // Solver
    // Create the mip solver with the CBC backend.
    MPSolver solver("AssignmentMip", MPSolver::CBC_MIXED_INTEGER_PROGRAMMING);

    // Variables
    // x[i][j] is an array of 0-1 variables, which will be 1
    // if worker i is assigned to task j.
    std::vector<std::vector<const MPVariable*>> x(num_workers, std::vector<const MPVariable*>(num_tasks));

    for (int i = 0; i < num_workers; ++i) {
        for (int j = 0; j < num_tasks; ++j) {
            x[i][j] = solver.MakeIntVar(0, 1, "");
        }
    }

    // Constraints
    // Each worker is assigned to at most one task.
    for (int i = 0; i < num_workers; ++i) {
        LinearExpr worker_sum;
        for (int j = 0; j < num_tasks; ++j) {
            worker_sum += x[i][j];
        }
        solver.MakeRowConstraint(worker_sum <= 1.0);
    }
    
    // Each task is assigned to exactly one worker.
    for (int j = 0; j < num_tasks; ++j) {
        LinearExpr task_sum;
        for (int i = 0; i < num_workers; ++i) {
            task_sum += x[i][j];
        }
        solver.MakeRowConstraint(task_sum == 1.0);
    }

    // Objective.
    MPObjective* const objective = solver.MutableObjective();
    for (int i = 0; i < num_workers; ++i) {
        for (int j = 0; j < num_tasks; ++j) {
            objective->SetCoefficient(x[i][j], costs[i][j]);
        }
    }
    objective->SetMinimization();

    // Solve
    const MPSolver::ResultStatus result_status = solver.Solve();

    // Print solution.
    // Check that the problem has a feasible solution.
    if (result_status != MPSolver::OPTIMAL & result_status != MPSolver::FEASIBLE) {
        std::cout << "No solution found.";
    }

    std::cout << "Total cost = " << objective->Value() << "\n\n";

    for (int i = 0; i < num_workers; ++i) {
        for (int j = 0; j < num_tasks; ++j) {
            // Test if x[i][j] is 0 or 1 (with tolerance for floating point
            // arithmetic).
            if (x[i][j]->solution_value() > 0.5) {
                std::cout << "Worker " << i << " assigned to task " << j
                          << ".  Cost = " << costs[i][j] << std::endl;
            }
        }
    }
}

Total cost = 265

Worker 0 assigned to task 3.  Cost = 70
Worker 1 assigned to task 2.  Cost = 55
Worker 2 assigned to task 1.  Cost = 95
Worker 3 assigned to task 0.  Cost = 45


## Solução CP-SAT

A seguir veja como resolver o problema usando o solucionador CP-SAT.

In [4]:
#include "ortools/sat/cp_model.h"
{
    // Data
    const std::vector<std::vector<double>> costs{
        {90, 80, 75, 70},   
        {35, 85, 55, 65},   
        {125, 95, 90, 95},
        {45, 110, 95, 115}, 
        {50, 100, 90, 100},
    };
    const int num_workers = costs.size();
    const int num_tasks = costs[0].size();

    // Model
    CpModelBuilder cp_model;

    // Variables
    // x[i][j] is an array of Boolean variables. x[i][j] is true
    // if worker i is assigned to task j.
    std::vector<std::vector<BoolVar>> x(num_workers, std::vector<BoolVar>(num_tasks));
    
    for (int i = 0; i < num_workers; ++i) {
        for (int j = 0; j < num_tasks; ++j) {
            x[i][j] = cp_model.NewBoolVar();
        }
    }

    // Constraints
    // Each worker is assigned to at most one task.
    for (int i = 0; i < num_workers; ++i) {
        LinearExpr worker_sum;
        for (int j = 0; j < num_tasks; ++j) {
            worker_sum.AddTerm(x[i][j], 1);
        }
        cp_model.AddLessOrEqual(worker_sum, 1);
    }
    
    // Each task is assigned to exactly one worker.
    for (int j = 0; j < num_tasks; ++j) {
        LinearExpr task_sum;
        for (int i = 0; i < num_workers; ++i) {
            task_sum.AddTerm(x[i][j], 1);
        }
        cp_model.AddEquality(task_sum, 1);
    }

    // Objective
    LinearExpr total_cost;
    for (int i = 0; i < num_workers; ++i) {
        for (int j = 0; j < num_tasks; ++j) {
            total_cost.AddTerm(x[i][j], costs[i][j]);
        }
    }
    cp_model.Minimize(total_cost);

    // Solve
    const CpSolverResponse response = Solve(cp_model.Build());

    // Print solution.
    if (response.status() == CpSolverStatus::INFEASIBLE) {
        std::cout << "No solution found.";
    }

    std::cout << "Total cost: " << response.objective_value() << std::endl;

    for (int i = 0; i < num_workers; ++i) {
        for (int j = 0; j < num_tasks; ++j) {
            if (SolutionBooleanValue(response, x[i][j])) {
                std::cout << "Task " << i << " assigned to worker " << j
                          << ".  Cost: " << costs[i][j] << std::endl;
            }
        }
    }
}

input_line_12:14:5: error: unknown type name 'CpModelBuilder'; did you mean 'operations_research::sat::CpModelBuilder'?
    CpModelBuilder cp_model;
    ^~~~~~~~~~~~~~
    operations_research::sat::CpModelBuilder
ortools/include/ortools/sat/cp_model.h:599:7: note: 'operations_research::sat::CpModelBuilder' declared here
class CpModelBuilder {
      ^
input_line_12:18:29: error: use of undeclared identifier 'BoolVar'
    std::vector<std::vector<BoolVar>> x(num_workers, std::vector<BoolVar>(num_tasks));
                            ^
input_line_12:18:86: error: expected a type
    std::vector<std::vector<BoolVar>> x(num_workers, std::vector<BoolVar>(num_tasks));
                                                                                     ^
input_line_12:18:86: warning: declaration does not declare anything [-Wmissing-declarations]
input_line_12:22:13: error: use of undeclared identifier 'x'
            x[i][j] = cp_model.NewBoolVar();
            ^
input_line_12:30:24: error: no m

Interpreter Error: 